In [20]:
import ipytest
ipytest.autoconfig()

Read numbers from the input:

In [21]:
%%ipytest -vv

def read_characters(input):
  return input.split(",")

def to_int(array):
  return [int(x) for x in array]

def read_numbers_from_file(file_path):
  with open(file_path) as f:
    for line in f:
      return to_int(read_characters(line))

def test_read_comma_separated_characters():
  input = "1,2,3,4"
  numbers = read_characters(input)
  assert numbers == ['1','2','3','4']

def test_convert_characters_to_numbers():
  input = ['1','2','3','4']
  numbers = to_int(input)
  assert numbers == [1,2,3,4]

def test_read_numbers_until_end_of_line():
  file = "day4-test1.input"
  numbers = read_numbers_from_file(file)
  assert numbers == [7,4,9,5,11,17]

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 3 items

tmp7hzuw1gp.py::test_read_comma_separated_characters PASSED                                  [ 33%]
tmp7hzuw1gp.py::test_convert_characters_to_numbers PASSED                                    [ 66%]
tmp7hzuw1gp.py::test_read_numbers_until_end_of_line PASSED                                   [100%]

======================================== 3 passed in 0.02s =========================================


Read bingo boards:

In [22]:
%%ipytest -vv

import re

def read_row(input):
  return [(x, False) for x in to_int(re.split(r'[ ]+', input.lstrip().rstrip('\n')))]

def read_all_boards(file_path):
  boards = []
  board = []
  line_counter = 0 
  with open(file_path) as f:
    for (idx, line) in enumerate(f):
      if idx < 2:
        continue

      if line == "\n":
        continue

      board.append(read_row(line))

      if (line_counter+1) % 5 == 0:
        boards.append(board)
        board = []
      
      line_counter += 1
        
  return boards

def test_read_row():
  input = " 5 84  3  7 87"
  row = read_row(input)
  assert row == [(5,False),(84,False),(3,False),(7,False),(87,False)]
  
def test_read_all_boards_from_file():
  file = "day4-test3.input"
  boards = read_all_boards(file)
  print(boards)
  assert boards == [
    [[(10, False),(27, False),(53, False),(91, False),(86, False)], 
    [(15, False),(94, False),(47, False),(38, False),(61, False)], 
    [(32, False),(68, False), (8, False),(88, False), (9, False)], 
    [(35, False),(84, False), (3, False), (7, False),(87, False)], 
    [(62, False),(78, False),(90, False),(66, False),(64, False)]],
    [[(30, False), (51, False), (26, False), (16, False), (57, False)],
    [(66, False), (88, False), (47, False), (75, False), (23, False)],
    [(61, False), (77, False), (64, False),  (9, False), (73, False)],
    [(44, False), (32, False), (28, False), (80, False), (81, False)],
    [ (3, False), (99, False), (67, False), (49, False), (78, False)]],
    [[(68, False), (92, False), (82, False), (74, False), (83, False)],
    [(12, False), (99, False), (80, False), (72, False),  (3, False)],
    [(56, False), (96, False), (36, False), (28, False), (43, False)],
    [(2, False),  (7, False), (14, False), (24, False),  (9, False)],
    [(63, False), (76, False), (40, False), (37, False), (73, False)]]
  ]



======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 2 items

tmpk6z_512m.py::test_read_row PASSED                                                         [ 50%]
tmpk6z_512m.py::test_read_all_boards_from_file PASSED                                        [100%]

======================================== 2 passed in 0.02s =========================================


Mark numbers on bingo boards and detect winner boards:

In [23]:
#1. for each number in the input
#2. for each board
#3. Mark if exists
#4. First board that has either all entries in horizontal or vertical direction marked is the winner

In [24]:
%%ipytest -vv

def mark_number(number, board):
  for (row_idx, row) in enumerate(board):
    for (col_idx, (nr,is_marked)) in enumerate(row):
      if nr == number and is_marked==False:
        board[row_idx][col_idx] = (nr, True)
        return True
  return False  

def play_bingo(numbers, boards):
  for number in numbers:
    for board in boards:
      mark_number(number, board)
      if is_winner(board):
        return calculate_score(number, board)

def is_winner(board):
  for row in board:
    if all([is_marked for (_, is_marked) in row]):
      return True

  column_length = len(board[0])
  for col_idx in range(column_length):
    if all([row[col_idx][1] for row in board]):
      return True

  return False

def calculate_score(last_number, board):
  return last_number * sum([number for row in board for (number, is_marked) in row if is_marked == False])

def test_is_winner_with_row_marked():
  file_path = "day4-test3.input"
  boards = read_all_boards(file_path)
  first_board = boards[0]
  first_board[1] = [(number, True) for (number, _) in first_board[1]]
  assert True == is_winner(first_board)

def test_is_winner_with_column_marked():
  file_path = "day4-test3.input"
  boards = read_all_boards(file_path)
  for row in boards[1]:
     for (idx, (number, _)) in enumerate(row):
      if idx == 1:
        row[idx] = (number,True)
  assert True == is_winner(boards[1])

def test_calculate_score():
  file_path = "day4-test3.input"
  boards = read_all_boards(file_path)
  first_board = boards[0]
  first_board[1] = [(number, True) for (number, _) in first_board[1]]
  last_number = first_board[1][2][0]
  score = calculate_score(last_number, first_board)
  assert score == 49256

def test_play_bingo():
  file_path = "day4-test4.input"
  numbers = read_numbers_from_file(file_path)
  boards = read_all_boards(file_path)
  result = play_bingo(numbers, boards)
  assert result == 4512

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 4 items

tmpyb82ez0u.py::test_is_winner_with_row_marked PASSED                                        [ 25%]
tmpyb82ez0u.py::test_is_winner_with_column_marked PASSED                                     [ 50%]
tmpyb82ez0u.py::test_calculate_score PASSED                                                  [ 75%]
tmpyb82ez0u.py::test_play_bingo PASSED                                                       [100%]

======================================== 4 passed in 0.03s =========================================


Solution part 1:

In [25]:
file_path = "day4.input"
numbers = read_numbers_from_file(file_path)
boards = read_all_boards(file_path)
result = play_bingo(numbers, boards)
print(result)

38594


In [26]:
%%ipytest -vv

def play_bingo_last_to_win(numbers, boards):
  winners = {}
  for number in numbers:
    for (idx, board) in enumerate(boards):
      if mark_number(number, board):
        if idx not in winners.keys() and is_winner(board):
          score = calculate_score(number, board)
          winners[idx] = score
  
  return [score for (_, score) in winners.items()][-1]

def test_last_to_win():
  file_path = "day4-test4.input"
  numbers = read_numbers_from_file(file_path)
  boards = read_all_boards(file_path)
  result = play_bingo_last_to_win(numbers, boards)
  assert result == 1924

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmpdy7xeb4_.py::test_last_to_win PASSED                                                      [100%]

======================================== 1 passed in 0.02s =========================================


Solution part 2:

In [28]:
file_path = "day4.input"
numbers = read_numbers_from_file(file_path)
boards = read_all_boards(file_path)
result = play_bingo_last_to_win(numbers, boards)
print(result)

21184
